# dataclass

## Что читать

* [https://docs.python.org/3/library/dataclasses.html](https://docs.python.org/3/library/dataclasses.html) - официальная документация
* [https://realpython.com/python-data-classes/](https://realpython.com/python-data-classes/)

## Зачем

Ничего нового, более короткое написание старого кода.

In [11]:
# подход через обычные классы
class RegularBook:
    def __init__(self, title, author):
        self.title = title
        self.author = author

    def __repr__(self):
        return f'{self.title=} {self.author}'

    def my_method(self):
        # какой-то метод с доступом к полям
        return self.title + self.author
        
rb = Book(title="Fahrenheit 451", author="Bradbury")
print(rb)

Book(title='Fahrenheit 451', author='Bradbury')


Через dataclass, значение типа обязательно, если не знаете - typing.Any

In [12]:
from dataclasses import dataclass

@dataclass
class Book:
    title: str
    author: str

    def my_method(self):
        return self.title + self.author

In [2]:
b1 = Book(title="Fahrenheit 451", author="Bradbury")
b1.author

'Bradbury'

In [3]:
print(b1)

Book(title='Fahrenheit 451', author='Bradbury')


In [4]:
b2 = Book(title="Fahrenheit 451", author="Bradbury")
b1 == b2

True

In [5]:
b3 = Book(title="Общая физика", author="Сивухин")
b1 == b3

False

Одинаково определение класса как

In [10]:
@dataclass
class C:
    ...

@dataclass()
class C:
    ...

@dataclass(init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False,
           match_args=True, kw_only=False, slots=False) # weakref_slot=False - новое
class C:
    ...

* `init`: если он равен True (по умолчанию), генерируется метод `__init__`. Если у класса уже определен метод `__init__`, параметр игнорируется.
* `repr`: включает (по умолчанию) создание метода `__repr__`. Сгенерированная строка содержит имя класса и название и представление всех полей, определенных в классе. При этом можно исключить отдельные поля (см. ниже)

* ``e`q: включает (по умолчанию) создание метоа ` __e`_`_. Объекты сравниваются так же, как если бы это были кортежи, содержащие соответствующие значения полей. Дополнительно проверяется совпадение типов
* order`er включает (по умолчанию выключен) создание метд`о`в _l`_``_`, _l`_``_`, _g`t__` `и _g`e`__. Объекты сравниваются так же, как соответствующие кортежи из значений полей. При этом так же проверяется тип объектов. Если order задан, а eq — нет, будет сгенерировано исключн`и`е Valuer`r`or. Так же, класс не должен содержать уже определенных методов сравне `
* `.
unsae_`h`ash влияет на генерацию ет`о`да _ha`s`h__. Поведение так же зависит от значений параетро`` `e`q fr`

* `frozen` - неизменяемые объекты, `FrozenInstanceError` при попытке изменить значение атрибута.

дописать: match_args=True, kw_only=False, slots=False) # weakref_slot=Falseozen

## Значения по умолчанию

На их основе делается конструктор, поэтому поля без значений по умолчанию должны идти сначала.

In [16]:
from dataclasses import dataclass, field
from typing import ClassVar

@dataclass
class TCase:
    # эта часть из xlsx TestIt
    id: int
    name: str
    automated: bool = False
    preconditions: str | None = None
    steps: list = field(default_factory=list)  # []  
    postconditions: str | None = None
    testdata: str = ''
    comments: str = ''
    iterations: str = ''
    priority: str = 'normal'
    severity: str = 'medium'
    state: str = 'draft'
    tags: list | None = None
    __id: ClassVar[int] = 20       # переменная класса, нужна, чтобы была сквозная нумерация тесткейсов


## Мутабельные значения по умолчанию

`steps: list = []` - ошибка, пишем через dataclasses.field

`steps: list = field(default_factory=list)`

## Атрибуты класса

`    __id: ClassVar[int] = 20`       - переменная класса

In [19]:
# asdict, astuple
from dataclasses import dataclass, asdict, astuple

@dataclass
class Point:
     x: int
     y: int

@dataclass
class C:
     mylist: list[Point]

p = Point(10, 20)
assert asdict(p) == {'x': 10, 'y': 20}

c = C([Point(0, 0), Point(10, 4)])
assert asdict(c) == {'mylist': [{'x': 0, 'y': 0}, {'x': 10, 'y': 4}]}

assert astuple(p) == (10, 20)
assert astuple(c) == ([(0, 0), (10, 4)],)

## Инициализация зависящих полей `__post_init__`

In [23]:
from dataclasses import dataclass

@dataclass
class C:
    a: float
    b: float
    c: float = field(init=False)

    def __post_init__(self):
        self.c = self.a + self.b

t = C(a=2.3, b=3.5)
print(t)

C(a=2.3, b=3.5, c=5.8)


In [26]:
class Rectangle:
    def __init__(self, height, width):
      self.height = height
      self.width = width

@dataclass
class Square(Rectangle):
    side: float

    def __post_init__(self):
        super().__init__(self.side, self.side)

sq = Square(2.5)
print(sq)

Square(side=2.5)


## Наследование в dataclass

In [12]:
from dataclasses import dataclass
from abc import ABC, abstractclassmethod

@dataclass
class A(ABC):
    _type: str | None = None
    x: int = 1
    text: str = ''

    def foo(self):
        print(f'y = {self.y}')   # это поле наследника

    @abstractclassmethod
    def myabs(self):
        pass


@dataclass
class B(A):
    y: int = 2
    answer: str = ''

    def __post_init__(self):
        self._type = 'text'

    def myabs(self):
        print(f'myabs: {self._type=} {self.x=} {self.y=}')

one = B(y=3, text='my text', x=7)
print(one)
one.foo()
one.myabs()

B(_type='text', x=7, text='my text', y=3, answer='')
y = 3
myabs: self._type='text' self.x=7 self.y=3


## Ответ на вопрос "Почему mutable объекты не надо ставить в значения по умолчанию"

In [36]:
class A:
    def __init__(self, steps=[], first='one'):
        self.steps = steps  # steps if steps else []
        self.steps.append(first)
        
    def __repr__(self):
        return repr(self.steps)

a = A(['qqq', 'bbb'], 'cc')
print(a)
b = A(first='cc')
print(b)
b2 = A(first='zzzz')
print(b2)

['qqq', 'bbb', 'cc']
['cc']
['cc', 'zzzz']
